In [1]:
import git
import pickle
from cottools.scc import collect_scc, SccData
from cottools.filesolver import NameRepo
# This import will be slow... Needs to be fixed.
from concernbert.frontend import CdCalculator
import csv
from datetime import datetime

In [2]:
def load_bytes(repo: git.Repo, hexsha: str) -> bytes:
    return repo.odb.stream(bytes.fromhex(hexsha)).read()  # type: ignore

In [3]:
scc_data = collect_scc("_repos/activemq")

# with open("_pickles/activemq_scc.pkl", "wb") as f:
#     pickle.dump(scc_data, f)

# with open("_pickles/activemq_scc.pkl", "rb") as f:
#     scc_data: dict[str, SccData] = pickle.load(f)

100%|██████████| 68806/68806 [00:12<00:00, 5621.16it/s]


In [4]:
#file_repo = NameRepo.parse_log("_repos/activemq", "HEAD")
# file_repo.dump("_pickles/activemq.pkl")
file_repo = NameRepo.load("_pickles/activemq.pkl")

In [5]:
cd_calculator = CdCalculator("_models/EntityBERT-v3_train_nonldl-lr5e5-2_83-e3", "_cache")

In [19]:
#name = "activemq-client/src/main/java/org/apache/activemq/ActiveMQConnection.java"
#name = "activemq-client/src/main/java/org/apache/activemq/ActiveMQMessageConsumer.java"
#name = "activemq-broker/src/main/java/org/apache/activemq/broker/BrokerService.java"
name = "activemq-broker/src/main/java/org/apache/activemq/broker/TransportConnection.java"
file_id = file_repo.file_id_by_name(file_repo.latest_commit(), name)

print(len(file_repo.changes_by_name(name)))
print(len(file_repo.changes_by_id(file_id)))

62
207


In [20]:
changes = file_repo.changes_by_id(file_id)
len(changes)
cont_changes = file_repo.cont_changes_by_id(file_id)
len(cont_changes)

207

In [21]:
repo = git.Repo("_repos/activemq", odbt=git.GitCmdObjectDB)

In [22]:
with open(f"decide_files/activemq-{file_id}.csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
# Write CSV header
        writer.writerow(['commit', 'commit_date', 'author_date', 'path', 'loc', 'lloc', 'comments', 'blanks', 'entities', 'inter_cd', 'intra_cd', 'cc'])
        for rev in file_repo.cont_changes_by_id(file_id):
            file_name = file_repo.file_name_by_id(rev, file_id)
            commit = repo.commit(rev)
            obj = commit.tree.join(file_name)
            if not isinstance(obj, git.Blob):
                raise RuntimeError
            content = load_bytes(repo,obj.hexsha)
            source = content.decode()
            cd = cd_calculator.calc_cd(source, pbar=False)
            print(cd.inter_cd)
            # print("Im HERE ")
            # print("Im HERE ")
            # print("Im HERE ")
            # print(source)
            #cd_calculator.calc_cd(source, pbar=True)
            loc = scc_data[obj.hexsha].loc
            blanks = scc_data[obj.hexsha].blanks
            complexity = scc_data[obj.hexsha].complexity
            
            # Now you can integrate this data into your CSV writing process:
            # Now integrate this data into your CSV writing process:
            writer.writerow([
                commit.hexsha,  # Commit Hash
                commit.committed_datetime,
                commit.authored_datetime,
                file_name,  # File Path
                loc,  # LOC (Lines of Code)
                loc,  # LLOC (Logical Lines of Code)
                commit.message.strip(),  # Comments
                blanks,  # Blanks 
                cd.num_entities, # num entities. 
                cd.inter_cd,  # Inter CD 
                cd.intra_cd,  # Intra CD 
                complexity  # Complexity from scc
            ])

1.19944966368419
1.1998329484714017
1.1873127591182406
1.1873127591182406
1.1873127591182406
1.1873127591182406
1.1921851525575367
1.2151814476352627
1.2160463407513955
1.2131510891511175
1.2126402047631755
1.2118984904210048
1.2178785520050952
1.2178785520050952
1.217189795473343
1.2152642788059622
1.2087298173262662
1.2098781186055767
1.223758932198571
1.2237508395693133
1.222156035170755
1.222156035170755
1.2240901859062434
1.222084132870146
1.2275357959406399
1.2275043435822568
1.2275630177694348
1.227689184277511
1.2808772672943596
1.2804281414473482
1.2789450941782146
1.278988359055748
1.2789241208279873
1.2798450577976477
1.2808279276405041
1.277808813511452
1.282412378658329
1.2824466870264721
1.2824018081233641
1.2823929513550725
1.2824111596970686
1.2827085406742065
1.2832695290420029
1.2834122665699315
1.283424729112297
1.2836733024402198
1.2831663085878269
1.2832407012662852
1.422523917907131
1.422477464055433
1.4223177212988043
1.4223107463200686
1.3828718577624963
1.38262